![](https://www.math.unipd.it/~marcuzzi/BannerStrumentifondamentali.png)

# Introduzione al Python GPU programming con Numba

## Che cos'è Numba?
- Numba è un **compilatore di funzioni Python** (sia per codice seriale su CPU che per codice parallelo su GPU)
- Numba è un compilatore **just-in-time**: le funzioni vengono tradotte non appena vengono chiamate 
- Numba è pensato per il **calcolo scientifico**: i dati devono essere di tipo int, float, o complex, inoltre Numba fornisce supporto built-in per il trasferimento di array Numpy tra CPU e GPU
- Numba accelera le funzioni generandone un'**implementazione per lo specifico tipo di dato** (int, float, ecc...) con cui stai lavorando, a differenza delle generiche funzioni Python che operano su tutti i tipi di dati
- Il compilatore di Numba produce un codice ottimizzato per la tua particolare CPU/GPU, perciò le prestazioni del tuo codice possono variare a seconda della macchina che utilizzi


NB: quando si programma in parallelo è importantissimo testare il proprio codice per assicurarsi che sia corretto!

In [1]:
import numpy as np
import math
from timeit import default_timer

In [2]:
import numba
from numba import jit, cuda, vectorize, guvectorize
from numba import void, uint8 , uint32, uint64, int32, int64, float32, float64, f8

print("numba", numba.__version__)

numba 0.39.0


In [3]:
print(cuda.detect())

Found 1 CUDA devices
id 0    b'GeForce GTX 1060 6GB'                              [SUPPORTED]
                      compute capability: 6.1
                           pci device id: 0
                              pci bus id: 1
Summary:
	1/1 devices are supported
True


In [4]:
my_gpu = cuda.get_current_device()
print("GPU in uso: ", my_gpu.name)
def cores_per_capability(cc): 
    if cc[0] == 3:
        return 192
    if cc[0] == 5:
        return 128
    if cc[0] == 6:
        if cc[1] == 0:
            return 64 
        else:
            return 128
    if cc[0] == 7:
        return 64
cc = my_gpu.compute_capability
print("Compute capability: ", "%d.%d" % cc, "(Numba richiede >= 3.0)")
print("Numero di Streaming Multiprocessors:", my_gpu.MULTIPROCESSOR_COUNT)
cores_per_multiprocessor = cores_per_capability(cc)
print("Numero di cores per SM:", cores_per_multiprocessor)
total_cores = cores_per_multiprocessor * my_gpu.MULTIPROCESSOR_COUNT
print("Numero di cores della GPU:", total_cores)

GPU in uso:  b'GeForce GTX 1060 6GB'
Compute capability:  6.1 (Numba richiede >= 3.0)
Numero di Streaming Multiprocessors: 10
Numero di cores per SM: 128
Numero di cores della GPU: 1280


# Tipi

I tipi di dati supportati da Numba sono i seguenti:

-   int
-   float
-   complex
-   bool
-   None
-   tuple

NB: **non** sono supportati gli oggetti.

Le seguenti funzioni built-in sono supportate:

-    abs()
-    bool
-    complex
-    enumerate()
-    float
-    int: solo la versione con un unico argomento 
-    len()
-    min(): solo la versione con argomenti multipli 
-    max(): solo la versione con argomenti multipli
-    range: viene riportato un oggetto range invece di un array di valori.
-    round()
-    zip()


Inoltre, sono supportate alcune funzioni dei moduli `math` (numeri reali) e `cmath` (numeri complessi). Per la lista completa vai su: https://numba.pydata.org/numba-doc/dev/cuda/cudapysupported.html

# Il decoratore @vectorize
>Converte una funzione tra scalari in una funzione vettoriale che agisce su di un array elemento per elemento. Molte delle funzioni di Numpy sono di questo tipo, come ad esempio `np.sin()`, `np.cos()`, `np.exp()`, e così via. 

La keyword target può prendere i seguenti valori
- cpu: crea una funzione seriale (ma ottimizzata) per la CPU
- parallel: crea una funzione parallela che viene eseguita su una multi-core CPU
- cuda: crea una funzione massivamente parallela per GPU

Noi ci concentreremo sul caso `target = cuda`. In questo caso, il compilatore Numba richiede che venga specificata la lista delle *segnature supportate dalla funzione*, cioè il tipo dei dati in input e in output. Una funzione che, ad esempio, prende in input due scalari `float32` e restutuisce un `float32` avrà segnatura `float32(float32, float32)`. Si posso specificare diverse segnature per otterene diverse copie della stessa funzione ottimizzate a seconda del tipo di dato. In questo caso le segnature vanno ordinate **da quella meno inclusiva a quella piu' inclusiva**.

Scriviamo una funzione che dati due array $x,y$ calcoli il vettore

$$ z[i] = sin(x[i])*exp(y[i]).$$

In [5]:
# Versione CPU
@numba.vectorize(['float32(float32, float32)',
                  'float64(float64, float64)'], target='cpu')
def sinexp_cpu(x, y):
    return math.sin(x) * math.exp(y)

# Versione CUDA
@numba.vectorize(['float32(float32, float32)',
                     'float64(float64, float64)'], target='cuda')
def sinexp_cuda(x, y):
    return math.sin(x) * math.exp(y)

In [6]:
# Generazione dei dati
n = 1000000
x = np.linspace(0, 2*np.pi, n)
y = np.linspace(10, 10, n)

In [7]:
# Verifica del risultato
np_ans = np.sin(x) * np.exp(y)
cpu_ans = sinexp_cpu(x, y)
cuda_ans = sinexp_cuda(x, y)

print("CPU vectorize e' corretto: ", np.allclose(cpu_ans, np_ans))
print("GPU vectorize e' corretto: ", np.allclose(cuda_ans, np_ans))

CPU vectorize e' corretto:  True
GPU vectorize e' corretto:  True


In [9]:
print("Tempo Numpy")
%timeit np.sin(x) * np.exp(y)

print("Tempo CPU vectorize")
%timeit sinexp_cpu(x, y)

print("Tempo GPU vectorize")
%timeit sinexp_cuda(x, y)

Tempo Numpy
47.1 ms ± 77.1 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Tempo CPU vectorize
41.7 ms ± 204 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Tempo GPU vectorize
6.85 ms ± 13.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
#speed-up
t_i = default_timer()
np.sin(x) * np.exp(y)
t_f = default_timer()
np_time = t_f -t_i

t_i = default_timer()
sinexp_cpu(x, y)
t_f = default_timer()
cpu_time = t_f -t_i

t_i = default_timer()
sinexp_cuda(x, y)
t_f = default_timer()
cuda_time = t_f -t_i

print('speed-up di CPU vectorize vs Numpy = ', np_time/cpu_time )
print('speed-up di GPU vectorize vs Numpy = ', np_time/cuda_time )

speed-up di CPU vectorize vs Numpy =  1.3654832173812843
speed-up di GPU vectorize vs Numpy =  12.613158023239503


## Cosa succede a basso livello?
- Trasferimenti in memoria automatizzati:
    gli array di Numpy vengono automatimente trasferiti in entrambe le direzioni
     - CPU -> GPU
     - GPU -> CPU
    Tuttavia è possibile gestire esplicitamente queste operazioni.
- Distribuzione del lavoro automatizzata:
    il lavoro viene distribuito in maniera automatica tra i processori della GPU.

- Gestione della memoria GPU automatizzata:
    la memoria della GPU viene allocata e liberata in maniera del tutto automatica.

## Gestire la memoria manualmente sulla GPU

Per allocare memoria sulla GPU ci sono due chiamate:
-  numba.cuda.device_array alloca senza inizializzare la memoria necessaria per un array con type e shape specifici (simile a numpy.empty)
-  numba.cuda.device_array_like alloca senza inizializzare la memoria necessaria per un array con type e shape di un altro array (simile a numpy.empty_like)
Se invece vogliamo trasferire sulla memoria del device un array CPU:
- numba.cuda.to_device crea una copia GPU di un array CPU

Queste chiamate restituiscono un oggetto Device Array

    class numba.cuda.cudadrv.devicearray.DeviceNDArray(shape, strides, dtype, stream=0, writeback=None, gpu_data=None)

Attraverso alcune chiamate ai metodi di quest'oggetto possiamo copiarlo in memoria CPU o GPU:
- GPU -> GPU / CPU -> GPU: self.copy_to_device(ary) copia ary in self. Se ary è un array GPU effettua un trasferimento device-to-device, se ary è un array CPU effettua un trasferimento host-to-device. 
- GPU -> CPU: self.copy_to_host(ary) copia self in ary se ary è un array CPU, crea un nuovo array Numpy se altrimenti ary è None.

Trasferiamo manualmente la memoria sulla GPU per vedere qual è il tempo di calcolo effettivo.

In [11]:
dz = cuda.device_array_like(x) # oppure numba.device_array((n,))
dx = cuda.to_device(x)
dy = cuda.to_device(y)

In [12]:
def check_pure_compute_time(dx, dy, dz):
    dz = sinexp_cuda(dx, dy)
    cuda.synchronize()   # assicura che il calcolo sia concluso
    
%timeit check_pure_compute_time(dx, dy, dz)

1.39 ms ± 880 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Il tempo di calcolo effettivo è minore di quello che avevamo misurato!
I trasferimenti in memoria sono molto costosi.

** Se si devono eseguire molti calcoli (ad. es. molto chiamate a funzioni vectorize), è meglio trasferire i dati manualmente un'unica volta prima delle chiamate. **

In [13]:
# copio in host memory
z = dz.copy_to_host()

In [14]:
del dx, dy, dz

# Il decoratore @guvectorize
> Il decoratore vectorize permette di scrivere funzioni che agiscono sui singoli elementi di un array, ma a volte questo è limitante. Il decoratore guvectorize permette di scrivere funzioni che lavorano su un numero arbitrario di elementi, e permette che gli array in input e output possano avere dimensioni diverse. Le funzioni compilate con guvectorize *non* ritornano alcun risultato, che inceve viene scritto riempendo un array che viene passato come **ultimo argomento in input**.


Il decoratore guvectorize inoltre richiede
- la lista ordinata delle segnature supportate (NB: dato che non ritornano alcun risultato, è necessario specificare solo il tipo dei dati in input)
- la dichiarazione simbolica delle dimensioni di input e output (NB: output = l'ultima variabile in input): ad esempio, una funzione che prende in input un array 1d di dimensione `n` e uno scalare e ritorna un array 1d di dimensione n si dichiarerà come `(n),()->(n)` (le parentesi vuote indicano lo scalare)

Scriviamo una funzione che data una matrice $A$ di dimensione $(n,m)$ ritorna il vettore $s$ di lunghezza $n$ definito da

$$ s[i] = \sum_{j = 1}^{m} A_{ij}$$


In [15]:
@guvectorize(['void(int32[:,:], int32[:])', 'void(float32[:,:], float32[:])'], '(n,m)->(n)', target='cuda')
def sum_row(A, s):
    for i in range(A.shape[0]):
        tmp = 0.
        for j in range(A.shape[1]):
            tmp += A[i,j]
        #endfor
        s[i] = tmp
    #endfor

In [16]:
n = 1000
a = np.random.random(n*n).astype(np.float32).reshape(n,n)

np_ans = np.sum(a, axis = 1)
gvec_ans = sum_row(a)

print("GPU guvectorize e' corretto:", np.allclose(np_ans, gvec_ans))

GPU guvectorize e' corretto: True


# Il decoratore @reduce
> Numba fornisce il decoratore reduce che converte un'operazione binaria in una funzione per la riduzione di array 1d.

Scriviamo una funzione che dato un vettore $x$ calcola
$$ s = \prod_i x[i].$$

In [17]:
@cuda.reduce
def prod_reduce(a, b):
    return a * b

In [18]:
# Generazione dei dati
n = 4000000
x = np.random.random(n) #(np.arange(n, dtype=np.float64)) + 1


np_ans = np.prod(x)      # riduzione numpy
gpu_ans = prod_reduce(x)   # riduzione cuda
print("GPU reduce e' corretto: ", np.allclose(np_ans, gpu_ans))

GPU reduce e' corretto:  True
